In [1]:
import geoplot as gplt 
import geoplot.crs as gcrs
import geopandas as gpd
gpd.__version__
import matplotlib.pyplot as plt
import pandas as pd
import pyproj
from fiona.crs import from_epsg
import mapclassify as mc
import glob
import os
#해안선 받아놓기
coastline= gpd.read_file('./Coastlinse_khoa/Coastlinse_khoa.shp', encoding='utf8')  


In [2]:
#맵 그리기 함수선언
def plot_map(data):
    ax = data.plot(figsize=(50, 30),facecolor='red', edgecolor='black',linewidth=0.5, zorder=2)
    coastline.plot(facecolor='#f6ebd4', edgecolor='black',ax=ax, zorder=1,linewidth=0.5)
    plt.grid(True, alpha=0.5, linestyle='--')
def plot_colormap(data, column, interval, data2=None, data3=None, figsize = [50,30]):
    if data2 is None:
        ax = data.plot(column, figsize=figsize,cmap = 'jet', linewidth=0, k=interval, legend=False)
        coastline.plot(facecolor='#f6ebd4', edgecolor='black',ax=ax, zorder=2,linewidth=0.5)
        plt.grid(True, alpha=0.5, linestyle='--')
    elif data3 is None:
        ax = data.plot(column, figsize=figsize,cmap = 'jet', linewidth=0, k=interval, legend=False)
        data2.plot(ax = ax, facecolor='black', edgecolor='black',linewidth=0.5)
        coastline.plot(facecolor='#f6ebd4', edgecolor='black',ax=ax, zorder=2,linewidth=0.5)
        plt.grid(True, alpha=0.5, linestyle='--')
    else:
        ax = data.plot(column, figsize=figsize,cmap = 'jet', linewidth=0, k=interval, legend=False)
        data2.plot(ax = ax, facecolor='black', edgecolor='black',linewidth=0.5)
        data3.plot(ax = ax, facecolor='red', edgecolor='black',linewidth=0.5)
        coastline.plot(facecolor='#f6ebd4', edgecolor='black',ax=ax, zorder=2,linewidth=0.5)
        plt.grid(True, alpha=0.5, linestyle='--')


In [3]:
MIN1_raw = gpd.read_file('./Ex/MIN1_DOWFSA_UTF8_mod/MIN1_DOWFSA_UTF8_mod.shp', encoding='euc-kr')  
MIN1 = MIN1_raw[['MIN1', 'geometry']]

In [4]:
flist = glob.glob('./Ex/220110/법정정보/**/*.shp', recursive=True)
for fname in flist:
    print(fname.split('\\')[2].split('.')[0])

골재채취단지
광업권
채굴권
탐사권
관광진흥관광지
마리나항만구역(면)
마리나항만구역(선)
마리나항만구역(점)
선상낚시포인트
수상레저
어촌체험마을
여객선터미널
해수욕장
해양레저활동구역
군사기지
군사보호구역
해상사격훈련구역
근해안강망어업
근해통발어업
근해형망어업
대형트롤어업
동해구중형트롤어업
선박안전교통 관제구역
수상레저금지구역
쌍끌이대형저인망어업
어선사고
어선원사고
연안정비기본계획
연안침식관리구역
연안침식실태조사
외끌이·쌍끌이대형저인망어업
중형저인망어업
침몰선박
해상교통관제센터
해저케이블
국가어항
마을어업
소규모어항
수산자원관리수면
수산자원보호구역
양식어업
어촌정주어항
정치망어업
정치성구획어업
지방어항
계측기
전원개발사업구역
해상풍력 발전사업허가
영해기점 무인도서
종합해양 과학기지
해양관측부이
교통안전특정해역
깊은수심항로
도선사승하선구역
마리나추천항로
무역항
신항만예정지역
양길항로
연안항
요트항로
일반항로
정박지구역
지방청 고시항로
지정항로
통항분리구역
통항제한금지구역
항로표지설치해역
항만구역
항만배후단지
국립공원
군립공원
도립공원
문화재구역
문화재보호구역
바다목장(면)
바다목장(점)
바다숲
생물권보전지역
생태경관보전지역
서식산란장(면)
서식산란장(점)
세계유산지정구역
습지보호구역
인공어초
자연환경보전지역
잘피숲
절대보전무인도서
준보전무인도서
천연기념물
천연보호구역
특정도서
해양보호구역
해양보호생물서식지
해중경관지구
환경보전해역


In [5]:
merge_data =MIN1
for fname in flist:
    print(fname.split('\\')[2].split('.')[0])
    try:
        data = gpd.read_file(fname, encoding='euc-kr')
    except Exception as e:
        print(e)
        gpd.read_file(fname, encoding='utf8')

    print(data.crs)
    data = data.to_crs(epsg=4326)
    print(data.crs)
    result = gpd.sjoin(MIN1, data,  op='intersects')
    result.drop_duplicates(['MIN1'], keep='first', inplace=True)

    # result.to_file('./result/ex6/shp/'+fname.split('\\')[2].split('.')[0]+'.shp', encoding='euckr')
    result[fname.split('\\')[2].split('.')[0]] = 1
    
    match = pd.merge(MIN1, result, on='MIN1', how='outer')
    match = match[['MIN1', fname.split('\\')[2].split('.')[0]]]

    merge_data = pd.merge(merge_data, match, on='MIN1', how='outer')
    # merge_data = pd.concat([merge_data, match], axis=1)
    # result.to_csv('./result/ex6/csv/'+fname.split('\\')[2].split('.')[0]+'.csv', index = False, encoding='euckr')
    # plot_map(result)
    # plt.savefig('./result/ex6/'+fname.split('\\')[1].split('.')[0]+'.png', dpi=300)
merge_data.to_file('./result/ex6/shp/MIN1_법정정보_2022.shp')
merge_data.drop('geometry', axis =1, inplace=True)
merge_data.to_csv('./result/ex6/csv/MIN1_법정정보_2022.csv', index = False, encoding='euckr')

골재채취단지
epsg:4326
epsg:4326
광업권
epsg:4326
epsg:4326
채굴권
epsg:4326
epsg:4326
탐사권
epsg:4326
epsg:4326
관광진흥관광지
epsg:4326
epsg:4326
마리나항만구역(면)
epsg:5179
epsg:4326
마리나항만구역(선)
epsg:5179
epsg:4326
마리나항만구역(점)
epsg:5179
epsg:4326
선상낚시포인트
epsg:5179
epsg:4326
수상레저
epsg:5179
epsg:4326
어촌체험마을
epsg:5179
epsg:4326
여객선터미널
epsg:5179
epsg:4326
해수욕장
epsg:4326
epsg:4326
해양레저활동구역
epsg:4326
epsg:4326
군사기지
epsg:4326
epsg:4326
군사보호구역
epsg:4326
epsg:4326
해상사격훈련구역
epsg:5179
epsg:4326
근해안강망어업
PROJCS["Transverse_Mercator",GEOGCS["GCS_Geographic_Coordinate_System",DATUM["WGS_1984",SPHEROID["WGS84",6378137,298.257223560493],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",38],PARAMETER["central_meridian",127.5],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",1000000],PARAMETER["false_northing",2000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
epsg:4326
근해통발어업
epsg

C:\Users\PPARK\AppData\Local\Temp/ipykernel_30608/779788139.py:27: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  merge_data.to_file('./result/ex6/shp/MIN1_법정정보_2022.shp')


SchemaError: Failed to create field name '골재채취단지': cannot convert to ISO-8859-1

In [31]:
merge_data

,MIN1,geometry,골재채취단지,광업권,채굴권,탐사권,관광진흥관광지,마리나항만구역(면),마리나항만구역(선),마리나항만구역(점),...,잘피숲,절대보전무인도서,준보전무인도서,천연기념물,천연보호구역,특정도서,해양보호구역,해양보호생물서식지,해중경관지구,환경보전해역
0,GR9_F1G11_A1,"POLYGON ((122.00000 39.00000, 122.01667 39.000...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GR9_F1G11_A4,"POLYGON ((122.00000 38.98333, 122.01667 38.983...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GR9_F1G11_A7,"POLYGON ((122.00000 38.96667, 122.01667 38.966...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GR9_F1G11_F1,"POLYGON ((122.00000 38.95000, 122.01667 38.950...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GR9_F1G11_F4,"POLYGON ((122.00000 38.93333, 122.01667 38.933...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421195,GR9_K2G44_T6,"POLYGON ((134.98333 30.08333, 135.00000 30.083...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421196,GR9_K2G44_T9,"POLYGON ((134.98333 30.06667, 135.00000 30.066...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421197,GR9_K2G44_Y3,"POLYGON ((134.98333 30.05000, 135.00000 30.050...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421198,GR9_K2G44_Y6,"POLYGON ((134.98333 30.03333, 135.00000 30.033...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
new.to_file('./result/ex6/shp/MIN1_2022.shp', encoding='utf8')

In [ ]:
merge_data[merge_data.duplicated(['MIN1']) ==True]

In [ ]:
merge_data[merge_data.duplicated(['MIN1']) ==True].to_csv('temp.csv', index = False, encoding='euckr')

In [ ]:
# merge_data['MIN1'].dropna(inplace=True)
merge_data[merge_data['MIN1'].isnull()==True]

In [ ]:
len(merge_data['MIN1'].unique().tolist())

In [ ]:
pd.merge(merge_data, match, on='MIN1', how='outer')

In [ ]:
match

In [ ]:
GRID_LIST = result['MIN1'].tolist()

In [ ]:
result

In [ ]:
match = pd.merge(MIN1, result, on='MIN1', how='outer')


In [ ]:
match

In [ ]:
match[fname.split('\\')[2].split('.')[0]] = 1
match = match[['MIN1', fname.split('\\')[2].split('.')[0]]]
match

In [ ]:
match[match['layer'].isnull() ==False]

In [ ]:
match.to_csv('20220110.csv', index = False, encoding='euckr')

In [20]:
from pyproj import CRS
crs_utm = CRS.from_user_input(4326)
crs_utm

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich